In [139]:
import random
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

In [140]:
def create_graphs(n,s,model,**kwargs):
    Graphs = []
    while len(Graphs) < n:
        G = model(s,**kwargs)
        while nx.is_connected(G) is False:
            G = model(s,**kwargs)
        Graphs.append(G)
    return Graphs

In [142]:
def lower_bound_optimal_solution(G,terms):
    shortest_path_bound = 0
    for i in range(len(terms)):
        for j in range(i + 1, len(terms)):
            path_length = nx.shortest_path_length(G, source=terms[i], target=terms[j], weight='weight')
            shortest_path_bound = max(shortest_path_bound, path_length)
    return shortest_path_bound

In [160]:
def average_lower_bound(graph_series,terms):
    total_bound = 0
    for G in graph_series:
        lb = lower_bound_optimal_solution(G, terms)
        total_bound += lb
    average_bound = total_bound / len(graph_series)
    return average_bound

In [208]:
def eval_sol(graph, terminals, sol) -> float:
    graph_sol = nx.Graph()
    for (i, j) in sol:
        if (i, j) in graph_sol.edges:
            graph_sol[i][j]['weight'] = 1 + graph_sol[i][j].get('weight', 1)
        else:
            graph_sol.add_edge(i, j, weight=graph[i][j].get('weight', 1))

    if not nx.is_tree(graph_sol):
        print ("Error: the proposed solution is not a tree")
        return -1

    for i in terminals:
        if not i in graph_sol:
            print ("Error: a terminal is missing from the solution")
            return -1

    cost = graph_sol.size(weight='weight')
    return cost

In [221]:
def generate_initial_steiner_tree(G, terminals):
    tree = nx.Graph()
    tree.add_nodes_from(terminals)

    for i in range(len(terminals)):
        for j in range(i + 1, len(terminals)):
            u = terminals[i]
            v = terminals[j]
            path = nx.shortest_path(G, source=u, target=v, weight='weight')
            nx.add_path(tree, path)
    if nx.is_tree(tree) and all(term in tree.nodes for term in terminals):
        return tree
    else:
        tree = nx.minimum_spanning_tree(tree, weight='weight')
        if nx.is_tree(tree) and all(term in tree.nodes for term in terminals):
            return tree
        else:
            return None



In [222]:
def rand_neighbor(G, current_solution, terminals):
    new_solution = current_solution.copy()
    
    operation = random.choice(["add_steiner_point", "prune_leaf"])
    if operation == "prune_leaf":
        # Remove a non-terminal leaf node
        leaf_nodes = [node for node in new_solution.nodes if new_solution.degree(node) == 1 and node not in terminals]
        if leaf_nodes:
            node_to_remove = random.choice(leaf_nodes)
            new_solution.remove_node(node_to_remove)
        
    elif operation == "add_steiner_point":
        # Add a non-terminal node to introduce a potential Steiner point
        non_terminal_nodes = set(G.nodes) - set(new_solution.nodes)
        if non_terminal_nodes:
            new_node = random.choice(list(non_terminal_nodes))
            connected = False
            attempt_limit = 20
            attempts = 0
            while not connected and attempts < attempt_limit:
                if len(new_solution.nodes) > 1:
                    node1, node2 = random.sample(list(new_solution.nodes), 2)
                    if not nx.has_path(new_solution, node1, node2):
                        if G.has_edge(node1, new_node) and G.has_edge(new_node, node2):
                            weight_1 = G.edges[node1, new_node].get('weight', 1)
                            weight_2 = G.edges[new_node, node2].get('weight', 1)
                            new_solution.add_node(new_node)
                            new_solution.add_edge(node1, new_node, weight=weight_1)
                            new_solution.add_edge(new_node, node2, weight=weight_2)
                            connected = True
                attempts += 1
                
    if nx.is_tree(new_solution) and nx.is_connected(new_solution) and all(term in new_solution.nodes for term in terminals):
        return new_solution
    else:
        return current_solution

In [223]:
def simulated_annealing(G, terms, T_init, T_end, nb_iter_per_temp):
    current_solution = generate_initial_steiner_tree(G, terms)
    if current_solution is None:
        raise ValueError("Failed to generate a valid initial Steiner tree.")
    current_weight = eval_sol(G, terms, current_solution.edges())
    best_solution = current_solution.copy()
    best_weight = current_weight
    k_max = 1000
    k = 0
    T = T_init
    while k < k_max and T > T_end:
        T = T_init * ((T_end / T_init) ** (k / k_max))
        for _ in range(nb_iter_per_temp):  
            new_solution = rand_neighbor(G, current_solution, terms)    
            new_weight = eval_sol(G, terms, new_solution.edges())

            # Accept new solution with probability based on temperature
            if new_weight < current_weight:
                proba = 1
            else:
                proba = np.exp((current_weight - new_weight) / T)
            if random.uniform(0, 1) < proba:
                current_solution = new_solution
                current_weight = new_weight
                if current_weight < best_weight:
                    best_solution = current_solution
                    best_weight = current_weight
                    
        k += 1

    return best_solution, best_weight


In [224]:
def average_solution(Graphs,terms,T_init=100,T_end=1,nb_iter_per_temp=50):
    total_weight = 0
    for G in Graphs: 
        _, weight = simulated_annealing(G, terms, T_init, T_end, nb_iter_per_temp)
        total_weight += weight
    average_weight = total_weight / len(Graphs)
    return average_weight

In [226]:
graph_series = create_graphs(50, 10, nx.generators.binomial_graph, p=0.5)
terminals = [0, 1, 2, 3, 4]
average_weight = average_solution(graph_series,terminals)
average_l_bound = average_lower_bound(graph_series, terminals)
print(f"Average Lower Bound: {average_l_bound}")
print(f"Average Steiner Tree weight: {average_weight}")

Average Lower Bound: 2.28
Average Steiner Tree weight: 4.5
